In [1]:
import warnings
warnings.filterwarnings('ignore')
    
import arviz as az
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
from gala.units import UnitSystem
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pymc3 as pm
import pymc3_ext as pmx
from tqdm.notebook import tqdm
import astropy.table as at
from aesara_theano_fallback import tensor as tt
import exoplanet.units as xu
import json

from yellowcard.model_pymc3 import TimingArgumentModelPymc3

In [2]:
plt.rcParams.update({"text.usetex":False})

### To get mass as a function of travel velocity

In [35]:
# # ------------------------------------------- #
# # this bit of the code runs pymc3 for various #
# # values of the travel velocity of the disk   #
# # ------------------------------------------- #
# # includes ~0.1% error on velocity measure    #
# # ------------------------------------------- #

# # modelChoice = "vdm2012"
# modelChoice = "fiducial2021"

# vtravel = 70
# vtravel_err = 1e-3

# if vtravel == 0:
#     model = TimingArgumentModelPymc3.from_dataset(f"../datasets/{modelChoice}.ecsv", 
#                                               include_vtravel=False)
# else:
#     model = TimingArgumentModelPymc3.from_dataset(f"../datasets/{modelChoice}.ecsv", 
#                                               include_vtravel=True,
#                                               vtravel_mag=vtravel,
#                                               vtravel_mag_err=vtravel_err)
# with model.pymc3_model:
#     start = {
#     'r': 770.,  
#     'M': 4.2, 
#     'eta': 4.5,
#     'ln(1-e)': -3,
#     'alpha': 0.5
#     }
#     res = pmx.optimize(start=start)
#     print(repr(res))

#     samples = pmx.sample(return_inferencedata=True, 
#                          start=res, tune=4000, draws=2000)

# samples.to_json(f"../pymc3-data/{modelChoice}_vtravel_{vtravel}.json")
# print(f"exported to ../pymc3-data/{modelChoice}_vtravel_{vtravel}.json")
# print("ALL. DAMN. DONE.")

optimizing logp for variables: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


message: Optimization terminated successfully.
logp: -3093.899065450101 -> 45.09710149881158


{'r_interval__': array(-2.63148973), 'M_interval__': array(-1.40578006), 'eta_angle__': array([-3.73934671, -2.45301572]), 'ln(1-e)_interval__': array(1.92396008), 'alpha_angle__': array([ 4.36693298, -0.96431133]), 'vtravel_mag': array(71.58985155), 'vtravel_lon': array(0.96429555), 'vtravel_lat': array(-0.59958543), 'r': array(764.67695625), 'M': array(4.33955984), 'eta': array(-2.15137872), 'sineta': array(-0.83614334), 'coseta': array(-0.54851099), 'ln(1-e)': array(-1.27420616), 'e': array(0.72034712), 'a': array(548.10903786), 'vscale': array(188.72234554), 'vrad': array(-81.4769659), 'vtan': array(93.82746352), 'tperi': array(13.74937331), 'sun_m31_dist': array(760.7340453), 'm31_icrs_xyz': array([561.8725182 , 106.01161022, 501.77395255]), 'alpha': array(1.78812977), 'model_pmra': array(0.23536552), 'model_pmdec': array(-0.18476058), 'model_rv': array(-308.86732915)}


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


Sampling 4 chains for 4_000 tune and 2_000 draw iterations (16_000 + 8_000 draws total) took 76 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There were 6 divergences after tuning. Increase `target_accept` or reparameterize.
There were 25 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


exported to ../pymc3-data/fiducial2021_vtravel_100.json
ALL. DAMN. DONE.


In [64]:
np.concatenate((np.array([32]),np.arange(0,105,5)))

array([ 32,   0,   5,  10,  15,  20,  25,  30,  35,  40,  45,  50,  55,
        60,  65,  70,  75,  80,  85,  90,  95, 100])

In [59]:
x

[32, 4, 3]

----
# Mass vs. vtravel

In [4]:
# ------------------------------------------- #
# this bit of the code runs pymc3 for various #
# values of the travel velocity of the disk   #
# ------------------------------------------- #
# includes ~0.1% error on velocity measure    #
# ------------------------------------------- #

modelChoices = ["fiducial2021","vdm2012"]

# travel velocities of MW disk wrt halo:
vtravels = np.concatenate( (np.arange(0,105,5), np.array([32])) )
vtravel_err = 1e-3

for dataset in modelChoices:
    print(f'working on {dataset}')
#     if dataset=="vdm2012":
#         continue
    for vel in vtravels:
        if vel == 0:
            model = TimingArgumentModelPymc3.from_dataset(f"../datasets/{dataset}.ecsv", 
                                                      include_vtravel=False)
        elif vel == 32:
            model = TimingArgumentModelPymc3.from_dataset(f"../datasets/{dataset}.ecsv", 
                                                      include_vtravel=True)
            
        else:
            model = TimingArgumentModelPymc3.from_dataset(f"../datasets/{dataset}.ecsv", 
                                                      include_vtravel=True,
                                                      vtravel_mag=vel,
                                                      vtravel_mag_err=vtravel_err)
        with model.pymc3_model:
            start = {
            'r': 770.,  
            'M': 4.2, 
            'eta': 4.5,
            'ln(1-e)': -3,
            'alpha': 0.5
            }
            res = pmx.optimize(start=start)
            print(repr(res))

            samples = pmx.sample(return_inferencedata=True, 
                                 start=res, tune=4000, draws=2000)

        samples.to_json(f"../pymc3-data/{dataset}_vtravel_{vel}.json")
print("ALL. DAMN. DONE.")

working on fiducial2021


optimizing logp for variables: [alpha, ln(1-e), eta, M, r]


message: Optimization terminated successfully.
logp: -36696.53165378691 -> 38.786484623869995


{'r_interval__': array(-2.63140118), 'M_interval__': array(-1.10862233), 'eta_angle__': array([-4.06223444, -1.87036129]), 'ln(1-e)_interval__': array(1.43361303), 'alpha_angle__': array([3.70109314, 2.51036045]), 'r': array(764.73186284), 'M': array(5.33849244), 'eta': array(-2.00228721), 'sineta': array(-0.90834323), 'coseta': array(-0.4182255), 'ln(1-e)': array(-1.92536356), 'e': array(0.85417727), 'a': array(563.44683879), 'vscale': array(206.45091363), 'vrad': array(-118.02080314), 'vtan': array(79.09496011), 'tperi': array(13.80100018), 'sun_m31_dist': array(760.78895427), 'm31_icrs_xyz': array([561.91307356, 106.01926203, 501.81017007]), 'alpha': array(0.9747999), 'model_pmra': array(0.24914964), 'model_pmdec': array(-0.1881097), 'model_rv': array(-308.85633316)}


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, ln(1-e), eta, M, r]


...

message: Optimization terminated successfully.
logp: -37085.75870154044 -> 38.29982038485984


{'r_interval__': array(-2.63132654), 'M_interval__': array(-1.29418061), 'eta_angle__': array([-3.92319368, -2.14675368]), 'ln(1-e)_interval__': array(1.58189616), 'alpha_angle__': array([4.40757687, 0.7571434 ]), 'vtravel_mag': array(32.75021741), 'vtravel_lon': array(0.97431392), 'vtravel_lat': array(-0.60141506), 'r': array(764.77815219), 'M': array(4.69534777), 'eta': array(-2.07148377), 'sineta': array(-0.87725278), 'coseta': array(-0.48002872), 'ln(1-e)': array(-1.70527107), 'e': array(0.81827688), 'a': array(549.0954401), 'vscale': array(196.12995553), 'vrad': array(-101.08374543), 'vtan': array(80.94525501), 'tperi': array(13.80098409), 'sun_m31_dist': array(760.83524562), 'm31_icrs_xyz': array([561.94726401, 106.02571293, 501.84070346]), 'alpha': array(1.40067451), 'model_pmra': array(0.24125096), 'model_pmdec': array(-0.18974999), 'model_rv': array(-308.86160855)}


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


Sampling 4 chains for 4_000 tune and 2_000 draw iterations (16_000 + 8_000 draws total) took 129 seconds.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
There were 82 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.9605988793773098, but should be close to 0.9. Try to increase the number of tuning steps.
The number of effective samples is smaller than 25% for some parameters.


working on vdm2012
ALL. DAMN. DONE.


---
# Mass error vs. pm error

In [10]:
0.001**-1

1000.0

In [9]:
x=1
x/=2
x

0.5

In [18]:
# ------------------------------------------- #
# this bit of the code runs pymc3 for various #
# values of the travel velocity of the disk   #
# ------------------------------------------- #
# includes ~0.1% error on velocity measure    #
# ------------------------------------------- #

modelChoices = ["fiducial2021","vdm2012"]
# pm_errs = np.array([1,1.5,2,4,10,20,50,75,100])

# pm_errs = np.array([1,10])
pm_errs = np.array([1,2,4,8,16])

for dataset in modelChoices:
    print(f'working on {dataset}')
#     if dataset == "fiducial2021":
#         continue
    for improv in pm_errs:
        model = (TimingArgumentModelPymc3.from_dataset(f"../datasets/{dataset}.ecsv", 
                                                      include_vtravel=True))
        
#         params = model_default.__dict__
#         print(params["pm_err"])
#         params["pm_err"] /= improv
#         print(params["pm_err"])
        
#         model = TimingArgumentModelPymc3(**params)
        
        with model.pymc3_model:
            pm.set_data({
            'obs_pmra_err':model.pm_err[0].to_value(model.units['angular speed'])/improv,
            'obs_pmdec_err':model.pm_err[1].to_value(model.units['angular speed'])/improv
            })
            start = {
            'r': 770.,  
            'M': 4.2, 
            'eta': 4.5,
            'ln(1-e)': -3,
            'alpha': 0.5
            }
            res = pmx.optimize(start=start)
            print(repr(res))

            samples = pmx.sample(return_inferencedata=True, 
                                 start=res, tune=4000, draws=2000)

        samples.to_json(f"../pymc3-data/{dataset}_pmimprov_{improv}.json")
        print(improv,np.std(samples.posterior.M))
print("ALL. DAMN. DONE.")

working on fiducial2021


optimizing logp for variables: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


message: Optimization terminated successfully.
logp: -37085.75870154044 -> 38.29982038485984


{'r_interval__': array(-2.63132654), 'M_interval__': array(-1.29418061), 'eta_angle__': array([-3.92319368, -2.14675368]), 'ln(1-e)_interval__': array(1.58189616), 'alpha_angle__': array([4.40757687, 0.7571434 ]), 'vtravel_mag': array(32.75021741), 'vtravel_lon': array(0.97431392), 'vtravel_lat': array(-0.60141506), 'r': array(764.77815219), 'M': array(4.69534777), 'eta': array(-2.07148377), 'sineta': array(-0.87725278), 'coseta': array(-0.48002872), 'ln(1-e)': array(-1.70527107), 'e': array(0.81827688), 'a': array(549.0954401), 'vscale': array(196.12995553), 'vrad': array(-101.08374543), 'vtan': array(80.94525501), 'tperi': array(13.80098409), 'sun_m31_dist': array(760.83524562), 'm31_icrs_xyz': array([561.94726401, 106.02571293, 501.84070346]), 'alpha': array(1.40067451), 'model_pmra': array(0.24125096), 'model_pmdec': array(-0.18974999), 'model_rv': array(-308.86160855)}


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


...

message: Optimization terminated successfully.
logp: -3385.3165603204884 -> 40.57879187584513


{'r_interval__': array(-2.63507715), 'M_interval__': array(-1.60551345), 'eta_angle__': array([-3.77415811, -2.39911032]), 'ln(1-e)_interval__': array(1.00859583), 'alpha_angle__': array([ 0.59916783, -4.43181652]), 'vtravel_mag': array(33.14398864), 'vtravel_lon': array(0.9652821), 'vtravel_lat': array(-0.58150898), 'r': array(762.45602404), 'M': array(3.76064265), 'eta': array(-2.13703003), 'sineta': array(-0.84392741), 'coseta': array(-0.53645738), 'ln(1-e)': array(-2.6725474), 'e': array(0.93092396), 'a': array(508.50706978), 'vscale': array(182.39650073), 'vrad': array(-95.56921125), 'vtan': array(44.42679564), 'tperi': array(13.74943497), 'sun_m31_dist': array(758.43075546), 'm31_icrs_xyz': array([560.17132543, 105.69063673, 500.25472142]), 'alpha': array(3.00721056), 'model_pmra': array(0.16613582), 'model_pmdec': array(-0.09804376), 'model_rv': array(-308.86258794)}


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


INFO (theano.gof.compilelock): Waiting for existing lock by process '51727' (I am process '51752')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/katiechambe/.theano/compiledir_macOS-10.15.7-x86_64-i386-64bit-i386-3.9.5-64/lock_dir
Sampling 4 chains for 4_000 tune and 2_000 draw iterations (16_000 + 8_000 draws total) took 55 seconds.


16 <xarray.DataArray 'M' ()>
array(0.32454385)
ALL. DAMN. DONE.


In [24]:
model = TimingArgumentModelPymc3.from_dataset(f"../datasets/{dataset}.ecsv", 
                                                      include_vtravel=True,
                                                      vtravel_mag=32,
                                                      vtravel_mag_err=1e-3)
with model.pymc3_model:
    start = {
    'r': 770.,  
    'M': 4.2, 
    'eta': 4.5,
    'ln(1-e)': -3,
    'alpha': 0.5
    }
    res = pmx.optimize(start=start)
    print(repr(res))

    samples = pmx.sample(return_inferencedata=True, 
                         start=res, tune=4000, draws=2000);
    
for name in ['M', 'r']:
    chain = np.asarray(getattr(samples.posterior, name)).ravel()
    print(f"{name}: {chain.mean():.2f} +/- {chain.std():.2f}")

optimizing logp for variables: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


message: Optimization terminated successfully.
logp: -2260.4941670852018 -> 45.072829861027785


{'r_interval__': array(-2.63133102), 'M_interval__': array(-1.29030243), 'eta_angle__': array([-3.91867302, -2.1549946 ]), 'ln(1-e)_interval__': array(1.58026439), 'alpha_angle__': array([4.40737222, 0.75833377]), 'vtravel_mag': array(32.72678928), 'vtravel_lon': array(0.97436866), 'vtravel_lat': array(-0.60140203), 'r': array(764.77537367), 'M': array(4.70813169), 'eta': array(-2.07358555), 'sineta': array(-0.87624193), 'coseta': array(-0.48187144), 'ln(1-e)': array(-1.70758042), 'e': array(0.81869606), 'a': array(548.42018359), 'vscale': array(196.51764549), 'vrad': array(-101.09456679), 'vtan': array(80.9216582), 'tperi': array(13.74967037), 'sun_m31_dist': array(760.83246699), 'm31_icrs_xyz': array([561.94521173, 106.02532571, 501.8388707 ]), 'alpha': array(1.40040443), 'model_pmra': array(0.24125054), 'model_pmdec': array(-0.18972886), 'model_rv': array(-308.86151893)}


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


INFO (theano.gof.compilelock): Waiting for existing lock by process '18098' (I am process '18123')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/katiechambe/.theano/compiledir_macOS-10.15.7-x86_64-i386-64bit-i386-3.9.5-64/lock_dir
Sampling 4 chains for 4_000 tune and 2_000 draw iterations (16_000 + 8_000 draws total) took 84 seconds.
There were 29 divergences after tuning. Increase `target_accept` or reparameterize.
There were 8 divergences after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.


M: 4.70 +/- 0.67
r: 764.58 +/- 10.89


In [25]:
model = TimingArgumentModelPymc3.from_dataset(f"../datasets/{dataset}.ecsv", 
                                                      include_vtravel=True,
                                                      vtravel_mag=32,
                                                      vtravel_mag_err=4)
with model.pymc3_model:
    start = {
    'r': 770.,  
    'M': 4.2, 
    'eta': 4.5,
    'ln(1-e)': -3,
    'alpha': 0.5
    }
    res = pmx.optimize(start=start)
    print(repr(res))

    samples = pmx.sample(return_inferencedata=True, 
                         start=res, tune=4000, draws=2000);
    
for name in ['M', 'r']:
    chain = np.asarray(getattr(samples.posterior, name)).ravel()
    print(f"{name}: {chain.mean():.2f} +/- {chain.std():.2f}")

optimizing logp for variables: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


message: Optimization terminated successfully.
logp: -2268.7882167253038 -> 36.77879750386076


{'r_interval__': array(-2.63133109), 'M_interval__': array(-1.29041216), 'eta_angle__': array([-3.91856573, -2.15518963]), 'ln(1-e)_interval__': array(1.58042578), 'alpha_angle__': array([4.40759734, 0.75702452]), 'vtravel_mag': array(32.75084106), 'vtravel_lon': array(0.97436383), 'vtravel_lat': array(-0.60140522), 'r': array(764.77532903), 'M': array(4.70776959), 'eta': array(-2.07363532), 'sineta': array(-0.87621794), 'coseta': array(-0.48191506), 'ln(1-e)': array(-1.70735189), 'e': array(0.81865462), 'a': array(548.41396286), 'vscale': array(196.51120286), 'vrad': array(-101.0822282), 'vtan': array(80.92641658), 'tperi': array(13.74967015), 'sun_m31_dist': array(760.83242234), 'm31_icrs_xyz': array([561.94517875, 106.02531949, 501.83884125]), 'alpha': array(1.40070148), 'model_pmra': array(0.24124424), 'model_pmdec': array(-0.18972762), 'model_rv': array(-308.8615226)}


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [vtravel_lat, vtravel_lon, vtravel_mag, alpha, ln(1-e), eta, M, r]


INFO (theano.gof.compilelock): Waiting for existing lock by process '18261' (I am process '18286')
INFO (theano.gof.compilelock): To manually release the lock, delete /Users/katiechambe/.theano/compiledir_macOS-10.15.7-x86_64-i386-64bit-i386-3.9.5-64/lock_dir
Sampling 4 chains for 4_000 tune and 2_000 draw iterations (16_000 + 8_000 draws total) took 81 seconds.
There were 8 divergences after tuning. Increase `target_accept` or reparameterize.
There were 34 divergences after tuning. Increase `target_accept` or reparameterize.
There were 7 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


M: 4.66 +/- 0.64
r: 764.47 +/- 10.78


___

___
# Graveyard ⚰️☠️

In [6]:
# truths = simulated_truths.copy()
# for k in list(samples.posterior.data_vars):
#     truths.setdefault(k, np.nan)
# truths = None    

In [7]:
# for name in ['M', 'r']:
#     chain = np.asarray(getattr(samples.posterior, name)).ravel()
#     print(f"{name}: {chain.mean():.2f} +/- {chain.std():.2f}")

In [8]:
# pcl = np.percentile(np.ravel(samples.posterior.M), [16, 50, 84])
# err1, err2 = np.diff(pcl)
# print(f"{pcl[1]:.3f} +{err1:.3f} -{err2:.3f}")

In [9]:
# for name in ['M', 'r','e']:
#     chain = np.asarray(getattr(samples.posterior, name)).ravel()
#     print(f"{name}: {chain.mean():.2f} +/- {chain.std():.2f}")